This is a notebook demonstrating the Transflower model from the paper [Transflower:Probabilistic autoregressive dance generation with multimodal attention](http://metagen.ai/transflower)

Transflower combines transformers with autoregressive normalizing flows, to do sequence modelling of high dimensional continuous signals. Here we apply it to dance generation, where we condition the generated motion on a piece of music.

To run this notebook simply run each cell in order, either by pressing ctrl+enter or by clicking the little play icons.


In [ ]:
#@title Connect Google Drive (double click to see code)
#@markdown This will connect colab to your Google Drive (will ask you for permission). Make sure you have more than a few hundred MB free on your Google Drive (to be safe)

#@markdown It will then create a folder to save transflower stuff (choose name below)

#@markdown or if it's already created it will use this name to find the folder

#@markdown If it asks for a code, click on the link, copy the code, paste it in the box, and press enter

#@markdown ---
#@markdown ### Enter a folder name:

from google.colab import drive, files
drive.mount('/content/drive')
import os
from os import mkdir, path
def cd(dir):
  os.chdir(dir)
transflower_folder_name = 'transflower_code' #@param {type:"string"}
working_dir=!pwd
working_dir=working_dir[0]
if working_dir=="/content":
  if not path.exists("drive/My Drive/"+transflower_folder_name):
    mkdir("drive/My Drive/"+transflower_folder_name)
  cd("drive/My Drive/"+transflower_folder_name)

#@markdown **run this cell** by pressing the play arrow on the left (or press ctrl+enter or shift+enter while having this cell selected)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Select which models to download (in the next step)
download_Transflower = True #@param {type:"boolean"}
download_Transflower_finetune = True #@param {type:"boolean"}
download_MoGlow = True #@param {type:"boolean"}
download_Transformer = True #@param {type:"boolean"}
download_Transflower_2nd_checkpoint = False #@param {type:"boolean"}

import os

os.environ["download_Transflower"] = str(download_Transflower)
os.environ["download_Transflower_finetune"] = str(download_Transflower_finetune)
os.environ["download_MoGlow"] = str(download_MoGlow)
os.environ["download_Transformer"] = str(download_Transformer)
os.environ["download_Transflower_2nd_checkpoint"] = str(download_Transflower_2nd_checkpoint)

In [ ]:
#@title Prepare transflower and dependencies
#@markdown Run this cell to download transflower to your drive (only will if it hasn't downloaded already), and prepare dependencies
%%capture
%%bash
if [[ -z "$(ls -A .)" ]]; then
  git clone https://github.com/guillefix/transflower-lightning .
  mkdir songs
  mkdir training/experiments
  if [ "$download_Transflower" == "True" ]; then
  gsutil -m cp -r gs://metagen/models/transflower_expmap_old training/experiments/
  fi
  if [ "$download_Transflower_finetune" == "True" ]; then
  gsutil -m cp -r gs://metagen/models/transflower_expmap_finetune2_old training/experiments/
  fi
  if [ "$download_Transflower_2nd_checkpoint" == "True" ]; then
  gsutil -m cp -r gs://metagen/models/transflower_expmap training/experiments/
  fi
  if [ "$download_Transformer" == "True" ]; then
  gsutil -m cp -r gs://metagen/models/transformer_expmap1 training/experiments/
  fi
  if [ "$download_MoGlow" == "True" ]; then
  gsutil -m cp -r gs://metagen/models/moglow_expmap training/experiments/
  fi
  gsutil -m cp -r gs://metagen/scalers/* songs/
  gsutil -m cp -r gs://metagen/seeds/* songs/
fi
pip install -r requirements.txt
pip install mido madgrad
apt-get install ffmpeg
sudo curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl
sudo chmod a+rx /usr/local/bin/youtube-dl

In [ ]:
#@title Get song
#@markdown Choose to download song from youtube, otherwise it asks you to upload file
download_from_youtube = True #@param {type:"boolean"}
youtube_url = 'https://www.youtube.com/watch?v=mE8YOyxJ3x4' #@param {type:"string"}
if download_from_youtube:
  !export youtube_url
  !youtube-dl -x --audio-format wav --restrict-filenames $youtube_url
  filename=!youtube-dl -x --audio-format mp3 --get-filename --restrict-filenames $youtube_url
  filename = os.path.splitext(filename[0])[0]+".wav"
  filename_new = filename[:50].replace(".","_")+".wav"
  !mv $filename songs/$filename_new
  filename = filename_new
  filename=filename[:-4] # removing final extensions
else:
  res=files.upload()
  filename=list(res.keys())[0]
  os.rename(filename,"songs/"+filename)
  # filename=filename[:-4] # removing final extensions

print(filename)

[youtube] TrlKxV5KWJo: Downloading webpage
[youtube] TrlKxV5KWJo: Downloading player 9da24d97
[download] Destination: 4_33-TrlKxV5KWJo.m4a
[download] 100% of 4.22MiB in 00:00
[ffmpeg] Correcting container in "4_33-TrlKxV5KWJo.m4a"
[ffmpeg] Destination: 4_33-TrlKxV5KWJo.wav
Deleting original file 4_33-TrlKxV5KWJo.m4a (pass -k to keep)
4_33-TrlKxV5KWJo_wav


In [ ]:
#@title Extract music features
#@markdown This will extract the features of the downloaded song(s)
%%capture
%%bash
chmod +x ./feature_extraction/audio_feature_extraction_test.sh
chmod +x ./feature_extraction/script_to_list_filenames
./feature_extraction/audio_feature_extraction_test.sh songs/

In [ ]:
#@title Generate dance 
#@markdown Generate dance ^^ 💃

#@markdown If it shows some nans ignore them <small>(its some issue if motion seed clip is shorter than the song)</small>

#@markdown You can choose one of the following motion seeds, that will affect the style (you can also input a custom one, but that's more advanced)
motion_seed = 'Casual' #@param ["Free style", "Casual", "Hip Hop", "Break dance", "Groovenet", "Casual2"]

#@markdown You can choose among these two models. If you want to try the baselines MoGlow or Transformer from the paper, you'd need to have downloaded them in the above step
model = 'Transflower' #@param ["Transflower", "Transflower fine tuned", "Transformer", "MoGlow", "Transflower_second_checkpoint"]
# model = 'Transflower' #@param ["Transflower", "Transflower fine tuned"] {allow-input: true}
generate_bvh = True #@param {type:"boolean"}
generate_video = True #@param {type:"boolean"}
short_test_run = False #@param {type:"boolean"}
seed_options = ["Free style", "Casual", "Hip Hop", "Break dance", "Groovenet", "Casual2"]
seed = str(seed_options.index(motion_seed) + 1)
!chmod +x ./script_generate.sh
!cp 'songs/seed'{seed}'.npy' 'songs/'{filename}'.expmap_scaled_20.npy'

if model == "Transflower":
  model_string = "transflower_expmap_old"
elif model == "Transflower fine tuned":
  model_string = "transflower_expmap_finetune2_old"
elif model == "MoGlow":
  model_string = "moglow_expmap"
elif model == "Transformer":
  model_string = "transformer_expmap1"
elif model == "Transflower_second_checkpoint":
  model_string == "transflower_expmap"
else:
  raise NotImplementedError("model "+model+" not implemented!")

extra_args = ""
if generate_bvh:
  extra_args += "--generate_bvh "
if generate_video:
  extra_args += "--generate_video "
if short_test_run:
  extra_args += "--max_length=512 "

!./script_generate.sh {model_string} {filename} {extra_args}--data_dir=songs;

chmod: cannot access './script_generate.sh': No such file or directory
cp: cannot stat 'songs/seed{seed}.npy': No such file or directory


The generated video (and bvh file if generated) are found inside `transflower_code/inference/generated/[model_name]/videos`

In [ ]:
#@title Visualize
#@markdown This will copy the generated dance to a server and open a visualizer
# %%capture
import uuid
from IPython.display import Javascript
hash = str(uuid.uuid1())
!echo $(echo {filename})_{hash}
!cp inference/generated/{model_string}/videos/{filename}.bvh inference/generated/{model_string}/videos/{filename}_{hash}.bvh
!gsutil cp inference/generated/{model_string}/videos/{filename}_{hash}.bvh gs://metagendance/
!cp inference/generated/{model_string}/videos/{filename}.mp4.mp3 inference/generated/{model_string}/videos/{filename}_{hash}.bvh.mp3
!gsutil cp inference/generated/{model_string}/videos/{filename}_{hash}.bvh.mp3 gs://metagendance/
url = 'https://guillefix.github.io/bvh_visualizer/?name='+filename+'_'+hash
print("Opening "+url)
display(Javascript('window.open("{url}");'.format(url=url)))

In [ ]:
# !cd ..; rm -r transflower_code